## Detect missing values

In [2]:
amount_missing_values_in_table = (X_train.isnull().sum())

# finding column names with missing values
NaN_columns =[i for i in X_train.columns
             if X_train[i].isnull().any()]
NaN_rows = [i for i in range(X_test.index[0],X_test.index[-1])
            if X_test.loc[i].isnull().any()]

NameError: name 'X_train' is not defined

## Replace missing values with mean

In [ ]:
#replace with mean
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
#put back column names
imputed_X_train.columns = X_train.columns

#Median, min, max ...
for i in names:
    X = X.fillna(X[i].median())
    #X[i + "_was_missing"] = X[i].isnull() (idicating changes)

#Perform RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=0)
model.fit(X, y)
preds = model.predict(X_test)

In [ ]:
#finding object columns for
#Label encoding
#One-Hot Encoding
s = (X_train.dtypes == "object")
object_columns = list(s[s].index)
or 
object_colsumns = [col for col in X_train.columns if X_train[col].dtype == "object"]

## Label encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Columns that can be safely label encoded
good_label_cols = [col for col in object_cols if 
                   set(X_train[col]) == set(X_valid[col])]
        
# Problematic columns that will be dropped from the dataset
bad_label_cols = list(set(object_cols)-set(good_label_cols))

# Drop categorical columns that will not be encoded
# Due to different unique values in train and validation set
label_X_train = X_train.drop(bad_label_cols, axis=1)
label_X_valid = X_valid.drop(bad_label_cols, axis=1)

# Apply label encoder 
label_encoder = LabelEncoder()
for col in good_label_cols:
    label_X_train[col] = label_encoder.fit_transform(label_X_train[col])
    label_X_valid[col] = label_encoder.fit_transform(label_X_valid[col])

# Get number of unique entries in each column with categorical data
object_nunique = list(map(lambda col: X_train[col].nunique(), object_cols))
d = dict(zip(object_cols, object_nunique))

# Print number of unique entries by column, in ascending order
sorted(d.items(), key=lambda x: x[1])

## One-Hot 

In [ ]:
#One_Hot for all with less than 10 unique values
low_cardinality_cols = [col for col in object_cols if X_train[col].nunique() < 10]
high_cardinality_cols = list(set(object_cols)-set(low_cardinality_cols))

from sklearn.preprocessing import OneHotEncoder

#define OH_encoder
OH_encoder = OneHotEncoder(handle_unknown="ignore", sparse = False)

# creat Df with One-Hot transformed low_cardinality_cols
One_Hot_encoded_list_train = pd.DataFrame(OH_encoder.fit_transform(X_train[low_cardinality_cols]))
One_Hot_encoded_list_valid = pd.DataFrame(OH_encoder.transform(X_valid[low_cardinality_cols])) 

#put index back
One_Hot_encoded_list_train.index = X_train.index
One_Hot_encoded_list_valid.index = X_valid.index

#copy
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

# creat Df without categorical variables 
rest_X_train = label_X_train.drop(X_train[object_cols],1)
rest_X_valid = label_X_valid.drop(X_valid[object_cols],1)

# concat lists
OH_X_train = pd.concat([rest_X_train,One_Hot_encoded_list_train],1)
OH_X_valid = pd.concat([rest_X_valid,One_Hot_encoded_list_valid],1)


## Mixing One-Hot  and Label encoding

In [ ]:
from sklearn.preprocessing import OneHotEncoder

#define OH_encoder
OH_encoder = OneHotEncoder(handle_unknown="ignore", sparse = False)

# creat Df with One-Hot transformed low_cardinality_cols
One_Hot_encoded_list_train = pd.DataFrame(OH_encoder.fit_transform(X_train[low_cardinality_cols]))
One_Hot_encoded_list_valid = pd.DataFrame(OH_encoder.transform(X_valid[low_cardinality_cols])) 

#put index back
One_Hot_encoded_list_train.index = X_train.index
One_Hot_encoded_list_valid.index = X_valid.index

#copy
label_X_train = X_train.copy()
label_X_valid = X_valid.copy()

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
#Label encode
for col in high_cardinality_cols:
    label_X_train[col] = label_encoder.fit_transform(X_train[col])
    label_X_valid[col] = label_encoder.transform(X_valid[col])
Label_encoded_train = label_X_train[high_cardinality_cols]
Label_encoded_valid = label_X_valid[high_cardinality_cols]

# creat Df without categorical variables 
rest_X_train = label_X_train.drop(X_train[object_cols],1)
rest_X_valid = label_X_valid.drop(X_valid[object_cols],1)

# concat lists
OH_X_train = pd.concat([rest_X_train,One_Hot_encoded_list_train,Label_encoded_train],1)
OH_X_valid = pd.concat([rest_X_valid,One_Hot_encoded_list_valid,Label_encoded_valid],1)

## Submission for Mixed One-Hot & Label-encoding

In [ ]:
from sklearn.preprocessing import OneHotEncoder
OH_encoder = OneHotEncoder(handle_unknown="ignore", sparse = False)

from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=0)

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

#delete NaN in X_test
X_test = start_X_test.copy()
X = start_X.copy()

containing_NaN = [i for i in range(X_test.index[0],X_test.index[-1]) if
                 X_test.loc[i].isnull().any()]

#upfilling form above
for i in containing_NaN:
    X_test.loc[i] = X_test.loc[i-1]


#set lists according do amount of NaN´s
object_cols = [col for col in X.columns if X[col].dtype == "object"]
low_cardinality_cols = [col for col in object_cols if X[col].nunique() < 10]
high_cardinality_cols = list(set(object_cols)-set(low_cardinality_cols))

# creat Df with One-Hot transformed low_cardinality_cols
One_Hot_encoded_list_train = pd.DataFrame(OH_encoder.fit_transform(X[low_cardinality_cols]))
One_Hot_encoded_list_valid = pd.DataFrame(OH_encoder.fit_transform(X_test[low_cardinality_cols])) 

#put index back
One_Hot_encoded_list_train.index = X.index
One_Hot_encoded_list_valid.index = X_test.index

#Label encode
for col in high_cardinality_cols:
    X[col] = label_encoder.fit_transform(X[col])
    X_test[col] = label_encoder.fit_transform(X_test[col])
Label_encoded_train = X[high_cardinality_cols]
Label_encoded_valid = X_test[high_cardinality_cols]

# create Df without categorical variables 
rest_X_train = X.drop(X[low_cardinality_cols],1)
rest_X_valid = X_test.drop(X_test[low_cardinality_cols],1)

# concat lists
final_X = pd.concat([rest_X_train,One_Hot_encoded_list_train,Label_encoded_train],1)
final_X_test = pd.concat([rest_X_valid,One_Hot_encoded_list_valid,Label_encoded_valid],1)

model.fit(final_X, y)
preds = model.predict(final_X_test)

output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds})
output.to_csv('submission.csv', index=False)

## Pipelines

In [ ]:
#selecting low categorial coumns / dropping high
categorical_cols = [cname for cname in X_train_full.columns if
                    X_train_full[cname].nunique() < 10 and 
                    X_train_full[cname].dtype == "object"]

#Selecting numerical columns
numerical_cols = [cname for cname in X_train_full.columns if 
                X_train_full[cname].dtype in ['int64', 'float64']]

#Those were dropped
high_ordinal = [i for i in X_train_full.columns
                if X_train_full[i].nunique() >= 10 and
               X_train_full[i].dtype == "object"]

print("Those rows were dropped: ",high_ordinal)

In [3]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

categorical_cols = [cname for cname in X_train_full.columns 
                    if  X_train_full[cname].nunique() < 10 and 
                        X_train_full[cname].dtype == "object"]

# Select numeric columns
numerical_cols = [cname for cname in X_train_full.columns 
                if X_train_full[cname].dtype in ['int64', 'float64']]

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='median, mean, constant, most_frequent')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Define model
model = RandomForestRegressor(n_estimators=100, random_state=0)

# Bundle preprocessing and modeling code in a pipeline
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('model', model)
                     ])

# Preprocessing of training data, fit model 
clf.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = clf.predict(X_valid)

print('MAE:', mean_absolute_error(y_valid, preds))

NameError: name 'X_train_full' is not defined

## Cross validation 

In [ ]:
from sklearn.model_selection import cross_val_score

# Multiply by -1 since sklearn calculates *negative* MAE
scores = -1 * cross_val_score(my_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')

## Optimizer and visualisation

In [ ]:
def get_score(n_estimators):
    my_pipeline = Pipeline(
        steps=[
            ('preprocessor', SimpleImputer()),
            ('model', RandomForestRegressor(n_estimators, random_state=0))
        ])
    scores = -1 * cross_val_score(my_pipeline, X, y,
                              cv=3,
                              scoring='neg_mean_absolute_error')
    return scores.mean()

List = {}
for i in range(50,401,50):
    List.update({i:get_score(i)})
    
results = List
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(list(results.keys()), list(results.values()))
plt.show()

## XGBoost

In [ ]:
from xgboost import XGBRegressor
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)